In [1]:
import sys
import gc
import os
import re
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
# import chromedriver_autoinstaller

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common. by import By

from bs4 import BeautifulSoup 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import konlpy
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter

import nltk

from wordcloud import WordCloud

In [331]:
gc.collect()

151

In [332]:
chrome_options = webdriver.ChromeOptions()

chrome_driver = 'C:/Users/jinho/Desktop/진호/국민대/3학년/22-1/텍스트데이터분석/5주차/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)

In [333]:
driver.get("https://www.musinsa.com/app/reviews/lists?type=&year_date=2022&month_date=6&day_date=&max_rt=2022&min_rt=2009&brand=&page=1&sort=new&hash_id=&best_type=&s_type=all&q=")

In [337]:
user_list = []
item_list = []
rating_list = []

In [338]:
gc.collect()

0

In [ ]:
for i in tqdm_notebook(range(3, 1002)): 
    gc.collect()
    driver = webdriver.Chrome(chrome_driver)
    time.sleep(1)
    # 일단 월별 추천순으로
    driver.get("https://www.musinsa.com/app/reviews/lists?type=&year_date=2022&month_date=6&day_date=&max_rt=2022&min_rt=2009&brand=&page={}&sort=new&hash_id=&best_type=&s_type=all&q=".format(i))
    time.sleep(1)
    for j in range(0, 20):
        # 유저 레벨 5이상인 경우
        if int(driver.find_elements_by_css_selector('div.review-list > div.review-profile > div.review-profile__text-wrap > div.review-profile__text > p.review-profile__name')[j].text.split(' ')[1]) >= 5:
            # 해당 유저명 변수 저장
            user_name = driver.find_elements_by_css_selector('div.review-list > div.review-profile > div.review-profile__text-wrap > div.review-profile__text > p.review-profile__name')[j].text
            # 해당 유저 이미지 클릭 -> 해당 유저가 남긴 모든 리뷰 확인 가능
            driver.find_elements_by_css_selector('div.review-list > div.review-profile > a')[j].click()
            time.sleep(1)
            for m in range(len(driver.find_elements_by_css_selector('div.nslist_bottom > div.pagination.textRight > div.wrapper > a'))-2):
                # 해당 유저의 모든 리뷰 수집
                for k in range(len(driver.find_elements_by_css_selector('div.review-list-wrap.review-list-wrap--member-review > div.review-list'))):
                    item_name = driver.find_elements_by_css_selector('div.review-list > div.review-goods-information > div.review-goods-information__item')[k].text.split('\n')[1]
                    rating = driver.find_elements_by_css_selector('div.review-list > div.review-list__rating-wrap > span.review-list__rating > span.review-list__rating__unit > span.review-list__rating__active')[k]
                    # rating 조건
                    if rating.size['width'] == 90:
                        rating = 5
                    elif rating.size['width'] == 72:
                        rating = 4
                    elif rating.size['width'] == 54:
                        rating = 3
                    elif rating.size['width'] == 36:
                            ating = 2
                    elif rating.size['width'] == 18:
                        rating = 1       

                    user_list.append(user_name)
                    item_list.append(item_name)
                    rating_list.append(rating)
                        
                if m == len(driver.find_elements_by_css_selector('div.nslist_bottom > div.pagination.textRight > div.wrapper > a')) -3:
                    time.sleep(1)
                    driver.get("https://www.musinsa.com/app/reviews/lists?type=&year_date=2022&month_date=6&day_date=&max_rt=2022&min_rt=2009&brand=&page={}&sort=new&hash_id=&best_type=&s_type=all&q=".format(i))
                    time.sleep(1)
                else:             
                    time.sleep(1)
                    driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[4]/div/div[1]/div/div/a[{}]'.format(m+2)).send_keys(Keys.ENTER)
                    time.sleep(1)
        else:
            pass


  0%|          | 0/999 [00:00<?, ?it/s]

In [323]:
df = pd.DataFrame({'유저명' : user_list, '아이템명': item_list, '평점' : rating_list})

In [339]:
df.유저명.nunique(), df.아이템명.nunique(), df.평점.nunique()

(9, 126, 3)

In [319]:
df.to_csv("한 부분까지.csv",index=False, encoding="utf-8-sig")

___

In [14]:
a = pd.read_csv('무신사 리뷰 크롤링_5월 ~559 .csv')
b = pd.read_csv('무신사 리뷰 크롤링_5월 560~ 804 .csv')
c = pd.read_csv('무신사 리뷰 크롤링_5월 805~ .csv')

In [15]:
df = pd.concat([a, b, c], axis = 0)

In [16]:
df = df.reset_index().iloc[:,1:]

In [17]:
df

,유저명,브랜드,아이템명,평점,리뷰
0,LV 3 qwrrtㅛ,와이케이,말티즈 아카이브 반팔티셔츠 브라운 women,5,티셔츠 진짜 예뻐요!!! 그리고 저희 집 강쥐랑 넘 닮아서 최애 티셔츠 될 것 같아요~!
1,LV 4 옒ㅣ,네스티팬시클럽,[NF] 캣 소프트 카라 니트 티 (BLUE)_F22QB417,5,저희집 탱자랑 똑같이 생겨서 맘에 들어요\n같이 사진도 찍었어요
2,LV 4 제리쀼,라퍼지스토어,[썸머] 린넨 크로켓 오버핏 니트 후드 집업 _Off White,3,화이트가 아닌데요.....?\n화이트가 아닌데 왜 화이트인척....
3,LV 5 subee77,라퍼지스토어,[SET]아일리쉬 린넨 세미 오버핏 싱글 자켓 벨트팬츠 셋업_Navy Blue,1,바지 양쪽 바깥쪽 다림질이 제대로 되지않고\n어디는 겹쳐서 되어있고 어디는 펼쳐서 ...
4,LV 4 뉴비_008ce7fdc8b0,무신사 스탠다드,우먼즈 튜브 브라 탑 [화이트],5,후기가 없을때 구매했는데 구매 가능했던 이유는오랫동안 찾던 디자인이여서요.. 인터넷...
...,...,...,...,...,...
20007,LV 3 뉴비_3945d3398f66,나이키,에어 테일윈드79 운동화 487754-012,5,좋아요 아들 운동화 구매했는데 디자인도 맘에 들고 만족합니다
20008,LV 3 뉴비_81a030cc2fa8,무신사 스탠다드,우먼즈 베이식 크루 넥 반팔 티셔츠 [화이트],5,깨끗한 흰티에요\n소재가 부들부들 너무 좋음\n비침도 덜함!
20009,LV 6 JayYun,엠엠지엘,미니멀 플립플랍 (블랙),2,발가락 끼는 부분은 일단 다 까진다보시면되고 발등 전체적으로 멍드는데 신을수가없네요...
20010,LV 5 cjappeifivujhdl,47브랜드,LA레이커스 NBA 볼캡 모자,5,보라색에 이어 또 샀네요. 물빠진 검정이 아니라 선명한 검정이라 더 좋아요. 레이커...


In [23]:
df.drop_duplicates()

,유저명,브랜드,아이템명,평점,리뷰
0,LV 3 qwrrtㅛ,와이케이,말티즈 아카이브 반팔티셔츠 브라운 women,5,티셔츠 진짜 예뻐요!!! 그리고 저희 집 강쥐랑 넘 닮아서 최애 티셔츠 될 것 같아요~!
1,LV 4 옒ㅣ,네스티팬시클럽,[NF] 캣 소프트 카라 니트 티 (BLUE)_F22QB417,5,저희집 탱자랑 똑같이 생겨서 맘에 들어요\n같이 사진도 찍었어요
2,LV 4 제리쀼,라퍼지스토어,[썸머] 린넨 크로켓 오버핏 니트 후드 집업 _Off White,3,화이트가 아닌데요.....?\n화이트가 아닌데 왜 화이트인척....
3,LV 5 subee77,라퍼지스토어,[SET]아일리쉬 린넨 세미 오버핏 싱글 자켓 벨트팬츠 셋업_Navy Blue,1,바지 양쪽 바깥쪽 다림질이 제대로 되지않고\n어디는 겹쳐서 되어있고 어디는 펼쳐서 ...
4,LV 4 뉴비_008ce7fdc8b0,무신사 스탠다드,우먼즈 튜브 브라 탑 [화이트],5,후기가 없을때 구매했는데 구매 가능했던 이유는오랫동안 찾던 디자인이여서요.. 인터넷...
...,...,...,...,...,...
20007,LV 3 뉴비_3945d3398f66,나이키,에어 테일윈드79 운동화 487754-012,5,좋아요 아들 운동화 구매했는데 디자인도 맘에 들고 만족합니다
20008,LV 3 뉴비_81a030cc2fa8,무신사 스탠다드,우먼즈 베이식 크루 넥 반팔 티셔츠 [화이트],5,깨끗한 흰티에요\n소재가 부들부들 너무 좋음\n비침도 덜함!
20009,LV 6 JayYun,엠엠지엘,미니멀 플립플랍 (블랙),2,발가락 끼는 부분은 일단 다 까진다보시면되고 발등 전체적으로 멍드는데 신을수가없네요...
20010,LV 5 cjappeifivujhdl,47브랜드,LA레이커스 NBA 볼캡 모자,5,보라색에 이어 또 샀네요. 물빠진 검정이 아니라 선명한 검정이라 더 좋아요. 레이커...


In [18]:
lst = list(df.리뷰.drop_duplicates().index)

In [19]:
len(lst)

18958

In [43]:
new_df = df.loc[lst].reset_index().iloc[:,1:]

In [24]:
df.to_csv('202205 리뷰.csv', index = False,  encoding="utf-8-sig")

3월 해야됌

## Concat

In [47]:
a = pd.read_csv('202201 리뷰.csv')
b = pd.read_csv('202202 리뷰.csv')
c = pd.read_csv('202203 리뷰.csv')
d = pd.read_csv('202204 리뷰.csv')
e = pd.read_csv('202205 리뷰.csv')

In [48]:
df = pd.concat([a, b, c, d, e], axis = 0)

In [49]:
df = df.reset_index().iloc[:,1:]

In [50]:
df = df.drop_duplicates()
df

,유저명,브랜드,아이템명,평점,리뷰
0,LV 3 김손희,굿라이프웍스,카라 케이블 니트 집업 아이보리,5,창균이가 오늘 심야아이돌에 나와서 이 옷을 입었어요.. 몇 주 전에 구입해서 잘 입...
1,LV 5 3우다,세터,카마로 집 데님 트러커 자켓 인디고,5,진짜 생지 여기만한 재질이 없습니다 핏이 너무 좋아요 크롭하고 오묘한 색감 정말 브...
2,LV 5 우빵하농,세터,카마로 집 데님 트러커 자켓 인디고,5,제가 31년 평생 데님자켓이 어울리지않아서 항상 사고 후회하고 버리거나 친구한테 줬...
3,LV 5 뉴비_6448bdb9,마뗑킴,ACCORDION WALLET IN BLACK,5,오랜 기다림 끝에 왔어요~♡\n너무 마음에 듬\n수납공간 넉넉하고 가죽 부드럽고 메...
4,LV 5 당신지금뭐하시주,시눈,MOUSSE KNIT BALACLAVA,4,색은 알록달록 예쁜데 쓰니까 얼굴에 닿으면 좀 따가워요 ㅠㅠ
...,...,...,...,...,...
97710,LV 3 뉴비_3945d3398f66,나이키,에어 테일윈드79 운동화 487754-012,5,좋아요 아들 운동화 구매했는데 디자인도 맘에 들고 만족합니다
97711,LV 3 뉴비_81a030cc2fa8,무신사 스탠다드,우먼즈 베이식 크루 넥 반팔 티셔츠 [화이트],5,깨끗한 흰티에요\n소재가 부들부들 너무 좋음\n비침도 덜함!
97712,LV 6 JayYun,엠엠지엘,미니멀 플립플랍 (블랙),2,발가락 끼는 부분은 일단 다 까진다보시면되고 발등 전체적으로 멍드는데 신을수가없네요...
97713,LV 5 cjappeifivujhdl,47브랜드,LA레이커스 NBA 볼캡 모자,5,보라색에 이어 또 샀네요. 물빠진 검정이 아니라 선명한 검정이라 더 좋아요. 레이커...


In [ ]:
df.reset_index().iloc[:,1:]

In [38]:
df = df.reset_index()

In [51]:
df.to_csv("2022 01~05.csv",index=False, encoding="utf-8-sig")

In [39]:
df

,index,유저명,브랜드,아이템명,평점,리뷰
0,0,LV 3 김손희,굿라이프웍스,카라 케이블 니트 집업 아이보리,5,창균이가 오늘 심야아이돌에 나와서 이 옷을 입었어요.. 몇 주 전에 구입해서 잘 입...
1,1,LV 5 3우다,세터,카마로 집 데님 트러커 자켓 인디고,5,진짜 생지 여기만한 재질이 없습니다 핏이 너무 좋아요 크롭하고 오묘한 색감 정말 브...
2,2,LV 5 우빵하농,세터,카마로 집 데님 트러커 자켓 인디고,5,제가 31년 평생 데님자켓이 어울리지않아서 항상 사고 후회하고 버리거나 친구한테 줬...
3,3,LV 5 뉴비_6448bdb9,마뗑킴,ACCORDION WALLET IN BLACK,5,오랜 기다림 끝에 왔어요~♡\n너무 마음에 듬\n수납공간 넉넉하고 가죽 부드럽고 메...
4,4,LV 5 당신지금뭐하시주,시눈,MOUSSE KNIT BALACLAVA,4,색은 알록달록 예쁜데 쓰니까 얼굴에 닿으면 좀 따가워요 ㅠㅠ
...,...,...,...,...,...,...
97454,97710,LV 3 뉴비_3945d3398f66,나이키,에어 테일윈드79 운동화 487754-012,5,좋아요 아들 운동화 구매했는데 디자인도 맘에 들고 만족합니다
97455,97711,LV 3 뉴비_81a030cc2fa8,무신사 스탠다드,우먼즈 베이식 크루 넥 반팔 티셔츠 [화이트],5,깨끗한 흰티에요\n소재가 부들부들 너무 좋음\n비침도 덜함!
97456,97712,LV 6 JayYun,엠엠지엘,미니멀 플립플랍 (블랙),2,발가락 끼는 부분은 일단 다 까진다보시면되고 발등 전체적으로 멍드는데 신을수가없네요...
97457,97713,LV 5 cjappeifivujhdl,47브랜드,LA레이커스 NBA 볼캡 모자,5,보라색에 이어 또 샀네요. 물빠진 검정이 아니라 선명한 검정이라 더 좋아요. 레이커...


In [40]:
p = df.groupby('유저명')['index'].count().reset_index()
p

,유저명,index
0,LV 1 Venusian,1
1,LV 1 Yum513,2
2,LV 1 zy123,1
3,LV 1 깜식이921,1
4,LV 1 날아라까마귀,2
...,...,...
56808,LV 8 흐에이에,2
56809,LV 8 흠청망청,1
56810,[STAFF] 네오센트로,1
56811,[STAFF] 현경최공,1


In [41]:
df = df.merge(p, on='유저명')

In [42]:
df

,index_x,유저명,브랜드,아이템명,평점,리뷰,index_y
0,0,LV 3 김손희,굿라이프웍스,카라 케이블 니트 집업 아이보리,5,창균이가 오늘 심야아이돌에 나와서 이 옷을 입었어요.. 몇 주 전에 구입해서 잘 입...,1
1,1,LV 5 3우다,세터,카마로 집 데님 트러커 자켓 인디고,5,진짜 생지 여기만한 재질이 없습니다 핏이 너무 좋아요 크롭하고 오묘한 색감 정말 브...,1
2,2,LV 5 우빵하농,세터,카마로 집 데님 트러커 자켓 인디고,5,제가 31년 평생 데님자켓이 어울리지않아서 항상 사고 후회하고 버리거나 친구한테 줬...,2
3,13549,LV 5 우빵하농,세터,카마로 벨티드 로우 데님,5,평소 허리 29 입습니다\nS사이즈 기준 빈틈없니 허리 따악 맞습니다.\n\n말그대...,2
4,3,LV 5 뉴비_6448bdb9,마뗑킴,ACCORDION WALLET IN BLACK,5,오랜 기다림 끝에 왔어요~♡\n너무 마음에 듬\n수납공간 넉넉하고 가죽 부드럽고 메...,1
...,...,...,...,...,...,...,...
97454,97704,LV 2 송사리살이,오에스티,[애플워치호환] 컬러링 스포츠 루프 스트랩 38-40mm/42-44mm (12종 택1),5,"포장도 꼼꼼하고, 색깔도 화면에서 보는거 그대로 입니다 ㅎㅎ 추가로 더 구매하려고요...",1
97455,97706,LV 4 ㅇㅇㅈ3232,토마스모어,TC1-TS0110 사커스웨트-올리버,5,루즈핏 맨투맨 찾고 있었는데 유연석님 입고 나온거 보고 브랜드 알게됐어요 허리 시보...,1
97456,97710,LV 3 뉴비_3945d3398f66,나이키,에어 테일윈드79 운동화 487754-012,5,좋아요 아들 운동화 구매했는데 디자인도 맘에 들고 만족합니다,1
97457,97711,LV 3 뉴비_81a030cc2fa8,무신사 스탠다드,우먼즈 베이식 크루 넥 반팔 티셔츠 [화이트],5,깨끗한 흰티에요\n소재가 부들부들 너무 좋음\n비침도 덜함!,1


In [46]:
df.query('index_y > ').reset_index().iloc[:,1:]

,index_x,유저명,브랜드,아이템명,평점,리뷰,index_y
0,43,LV 5 새벽이슬1004,티메이커,[GN]TMR-MG006 [맨투맨],3,옷도 맘에 들고 두께감도 맘에 들고 사이즈감도 좋습니다. 빈티지한 멋도 있고 잘 입...,13
1,13049,LV 5 새벽이슬1004,배드인배드,COLOR BLOCK FLEECE JACKET_BLACK,4,너무 맘에들어요 옷 코디하기도 좋습니다. 이번 겨울 이 옷으로 마무리 하지 않을까 ...,13
2,13069,LV 5 새벽이슬1004,원헌드레드퍼센트,OHP X MOAI b어쎄신and걸 베이직 맨투맨,2,너무 옷이 잘 늘어날 것 같습니다. 그림만 맘에 드네요~,13
3,13078,LV 5 새벽이슬1004,커버낫,스몰 어센틱 로고 후디 집업 피그먼트오렌지,3,커버낫 일반 후드티가 맘에들어 집업도 하나더 구매를 하게 되었네요~ ^^ 잘 입도록...,13
4,13086,LV 5 새벽이슬1004,그레이버,[UNISEX]엘보우 스마일드로잉 클립 맨투맨_블랙,2,생각보단 옷이 별로인 것 같습니다 팔꿈치에만 포인트가 있는데 이건 뭐 그냥 검정색티...,13
...,...,...,...,...,...,...,...
4807,97638,LV 4 aecpa,커버낫,C 로고 맨투맨 네이비,5,셔츠안에 레이어드해서 입으니까 확 사네요 너무예뻐요 제가 딱원하는 핏!,11
4808,97647,LV 4 aecpa,로맨틱크라운,리센츠 로고 니트 베스트_딥 그린,5,셔츠랑 세트로 잘입었어요 니트 예쁘다는소리 많이들음,11
4809,97667,LV 4 aecpa,로맨틱크라운,리센츠 로고 옥스포드 셔츠_화이트,5,평소에 XL입어서 샀다가 너무커서 기겁하고 바로 L로 교환했어요\n셔츠는 역시 로크 ~!,11
4810,97684,LV 4 aecpa,테이크이지,케이블 스몰라쿠 니트 베스트(네이비),5,폴로아닌데도 폴로같은느낌나고 꽈배기 떨어진게 너무 예뻐요 갠적으로 안에 셔츠입는것보...,11


In [38]:
df.query('브랜드_y > 2')

,유저명,브랜드_x,아이템명,평점,리뷰,브랜드_y
14,LV 6 땡떙떙,나이키,에어맥스 던 운동화 DJ3624-002,3,신발 받았는데 누가 반품한거를 보냈나 택도 없는 플라스틱만 달려있는ㅋㅋ어이가 없는데...,5
15,LV 6 땡떙떙,어나더 오피스,Layer Donegal Vest (Donegal Gray),4,어나더 오피스 긴팔 가디건 사고 잘 입고 있어서 베스트도 주문~! 좋아요,5
16,LV 6 땡떙떙,비얼디드키드,밴딩 실루엣 팬츠 블랙,5,네이비 사고 바로 블랙 주문ㅋㅋ떨어지는 핏이 예쁜 바지,5
17,LV 6 땡떙떙,에이전트픽,AGP 오가닉 크롭 가디건 라이트 베이지,5,입었을 때 핏이 예쁜데 재질도 좋아용 에이전트픽 굿~~~,5
18,LV 6 땡떙떙,에이전트픽,AGP 로고 티셔츠 핑크,5,재질도 좋고 색감도 예쁘고 에이전트픽 색감 맛집~~,5
...,...,...,...,...,...,...
77559,LV 4 카카오라이언,디스이즈네버댓,TNT HOKA T-Shirt Black,5,잘닙고다녀요 포인트주기좋고 귀여워요 여름에도 잘입을듯,3
77560,LV 4 카카오라이언,로커드,조거 스웻팬츠 (그레이),5,잘입고다닙니다 독서실갈때 편해서 좋네요 얇은거도 있으면 좋을듯,3
77572,LV 4 890723,예스아이씨,2Pack Coolmax C-logo Tee,4,생각보다 총장이 너무 길어서 조금 아쉬워요\n세일할때 싸게 사서 만족해요,3
77573,LV 4 890723,예일,YALE 1930 TEE NAVY,5,기본티로 좋은 것 같아요 세일할때 싸게 사서 만족해요,3


In [40]:
df.query('브랜드_y > 3')

,유저명,브랜드_x,아이템명,평점,리뷰,브랜드_y
14,LV 6 땡떙떙,나이키,에어맥스 던 운동화 DJ3624-002,3,신발 받았는데 누가 반품한거를 보냈나 택도 없는 플라스틱만 달려있는ㅋㅋ어이가 없는데...,5
15,LV 6 땡떙떙,어나더 오피스,Layer Donegal Vest (Donegal Gray),4,어나더 오피스 긴팔 가디건 사고 잘 입고 있어서 베스트도 주문~! 좋아요,5
16,LV 6 땡떙떙,비얼디드키드,밴딩 실루엣 팬츠 블랙,5,네이비 사고 바로 블랙 주문ㅋㅋ떨어지는 핏이 예쁜 바지,5
17,LV 6 땡떙떙,에이전트픽,AGP 오가닉 크롭 가디건 라이트 베이지,5,입었을 때 핏이 예쁜데 재질도 좋아용 에이전트픽 굿~~~,5
18,LV 6 땡떙떙,에이전트픽,AGP 로고 티셔츠 핑크,5,재질도 좋고 색감도 예쁘고 에이전트픽 색감 맛집~~,5
...,...,...,...,...,...,...
77526,LV 5 야쿠다,무신사 스탠다드,멀티 스트라이프 크루 넥 반팔 티셔츠 TYPE 4,5,생각보다 핏도 색감도 이쁩니다 믿고사는 무탠입니다,6
77542,LV 3 kwake,무신사 스탠다드,쿨탠다드 릴렉스 핏 크루 넥 반팔 티셔츠 [라이트 카키],5,얇고 시원함\n생각보다 주름잏많이져서 다려입어야할듯,4
77543,LV 3 kwake,무신사 스탠다드,코튼 이지 쇼츠 [카키],5,얇지만 탄탄하고 시원한 재질임\n약간 바스락거림,4
77544,LV 3 kwake,멜란지 마스터,피그먼트 스웨트 팬츠 [네이비]_와펜 ver.,5,운동복으로 막입기에 편함\n자주입고 여름에는 입기에 더울듯,4
